In [1]:
import os


In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories=['with_mask','without_mask']

In [5]:
data=[]
for category in categories:
    path=os.path.join('C:\\Users\\pushk\\face_mask_detector\\train',category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path=os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [6]:
#we have 755 with mask and 753 without mask
len(data)

1508

In [7]:
#shuffle so that the model doesnot develop a bias
import random
random.shuffle(data)

In [8]:
x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)

In [9]:
len(x)

1508

In [10]:
import numpy as np

x=np.array(x)
y=np.array(y)

x.shape

(1508, 224, 224, 3)

In [11]:
y.shape

(1508,)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [13]:
x_test.shape

(302, 224, 224, 3)

In [14]:
x_train.shape

(1206, 224, 224, 3)

In [15]:
from keras.applications.vgg16 import VGG16

In [16]:
vgg=VGG16()

In [17]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
#remove the last layer
from keras import Sequential
model = Sequential()

for layer in vgg.layers[:-1]:
    model.add(layer)
  

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [20]:
#now freeze the layers 
for layer in model.layers:
    layer.trainable=False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
from keras.layers import Dense

In [22]:
model.add(Dense(1,activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))

Epoch 1/2
38/38 [==============================] - 304s 8s/step - loss: 0.0908 - accuracy: 0.9701 - val_loss: 0.0430 - val_accuracy: 0.9901
Epoch 2/2
38/38 [==============================] - 296s 8s/step - loss: 0.0147 - accuracy: 0.9950 - val_loss: 0.0288 - val_accuracy: 0.9901


In [27]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    y_pred = np.round(y_pred).astype(int)
    return y_pred[0][0]

In [28]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0] + text_size[0][0] + 2
    end_y=pos[0] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    

In [29]:
haar = cv2.CascadeClassifier('C:\\Users\\pushk\\face_mask_detector\\haarcascade_frontalface_default.xml')

In [30]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [35]:
import cv2
cap=cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img = cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    
    if y_pred==0:
        draw_label(frame,"yes mask ",(30,30),(0,255,0))
    else:
        draw_label(frame,"no mask ",(30,30),(0,0,255))
        
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
    
cv2.destroyAllWindows()

1/1 [==============================] - 0s 219ms/step


1/1 [==============================] - 0s 269ms/step


1/1 [==============================] - 0s 258ms/step


1/1 [==============================] - 0s 251ms/step


1/1 [==============================] - 0s 247ms/step
